### Для входа нужны
- attend.csv
- groups.csv
- uniq_streets_coords.csv
- _users_features.csv

### Выход
- _attend_features.csv

In [1]:
import numpy as np
import pandas as pd

### Читаем данные

In [2]:
df_attend = pd.read_csv('attend.csv')
if 'Unnamed: 0' in df_attend.columns:
    df_attend.drop(columns=['Unnamed: 0'], inplace=True)
display(df_attend)

### Группируем отдельные занятия посещения человеком, формируем новый столбец с количеством посещений

In [3]:
df_attend.drop(columns = ['уникальный номер группы'],inplace =True)

tmp = df_attend.groupby(['уникальный номер участника', 'направление 3'], as_index=False).size()

df_attend = tmp.merge(
    df_attend.drop_duplicates(subset=['уникальный номер участника', 'направление 3'], keep='last'),
    left_on=['направление 3','уникальный номер участника', ],
    right_on=['направление 3','уникальный номер участника', ], how='left')

df_attend = df_attend.drop(columns=['дата занятия','уникальный номер занятия'])
display(df_attend)

### Вычисляем длительность занятия

In [4]:
df_attend['lesson_length'] = (df_attend['время окончания занятия'].astype('datetime64[ns]') - df_attend['время начала занятия'].astype('datetime64[ns]'))
df_attend['lesson_length'] = df_attend['lesson_length'] / pd.Timedelta('1 hour')

### Добавляем новое поле - часть дня занятия (утро, день, вечер)

In [5]:
b = [9,13,17,23]
l = ['mornig','aftennoon','evening']
df_attend['session'] = pd.cut(pd.to_datetime(df_attend['время начала занятия']).dt.hour, bins=b, labels=l, include_lowest=True)

### Удаляем ненужные столбцы

In [6]:
df_attend = df_attend.drop(columns=['время начала занятия','время окончания занятия', 'направление 2'])

### Логарифмируем количество посещенией занятия юзером (уменьшаем разброс между большим и малым количеством занятий на разных курсах)

In [7]:
df_attend['size'] = np.log(df_attend['size'])

### Переименовываем стоблцы на английский для удобства

In [8]:
df_attend = df_attend.rename(columns={
    'size':'log_visits',
    "уникальный номер участника": "user_id",
    'онлайн/офлайн':'is_online'})

In [9]:
df_attend.replace(('Да', 'Нет'), ('online', 'offline'), inplace=True)

## Обогащаем данными из таблицы с группами

In [10]:
df_groups = pd.read_csv('groups.csv')
df_groups = df_groups[['направление 3','направление 1','направление 2','адрес площадки']]
display(df_groups)

### Вычисляем координаты занятия на основе адреса

In [11]:
# Выделяем улицу
def get_street(address):
    try:
        start = address.lower().find('москва')
        address = address[start+7:]
        address = address[:address.find(',')]
    except:
        address = 'москва'
    address = address.replace('улица','').strip().lower()
    return address

df_groups['group_address'] = df_groups['адрес площадки'].apply(get_street)



### Читаем таблицу улиц-координат

In [12]:
mid = '(55.7498, 37.5371)' # Башня Федерация

# Получаем координаты
df_uniq_streets_coords = pd.read_csv('uniq_streets_coords.csv')
df_uniq_streets_coords = df_uniq_streets_coords[['для координат','Coords']]
df_uniq_streets_coords['для координат']= df_uniq_streets_coords['для координат'].str.lower()
df_uniq_streets_coords['Coords'].fillna(mid, inplace=True)

streets_dict = dict(zip(df_uniq_streets_coords['для координат'], df_uniq_streets_coords['Coords']))

In [13]:
df_groups.drop_duplicates(subset=['направление 1', 'направление 2', 'направление 3'], keep='last', inplace=True)

In [14]:
display(df_groups)

### Добавляем координаты занятия (на основе адреса улицы)

In [15]:
def match_street_coords(street):  
    res = []
    for i in streets_dict.keys():
        if(i.find(street) != -1):
            res.append(streets_dict[i])    
    if len(res) > 0:
        return res[0]
    else:
        return '(55.7498, 37.5371)'
split_data = df_groups['group_address'].apply(match_street_coords).str.strip(')').str.strip('(').str.split(',')
df_groups['lat'] = split_data.apply(lambda x: x[0]).astype(float)
df_groups['long'] = split_data.apply(lambda x: x[1]).astype(float)
df_groups = df_groups.drop(columns=['адрес площадки'])
display(df_groups)

### Добавляем в attend столбцы направление 1, направление 2

In [16]:
df_attend = df_attend.merge(
    df_groups,
    left_on=['направление 3'], 
    right_on=['направление 3'],
    how='left')

In [17]:
display(df_attend)

## Обогащаем данными из таблицы с пользователями

In [18]:
df_users = pd.read_csv('_users_features.csv')
df_users.drop(columns=['Unnamed: 0'],inplace=True)
display(df_users)

In [19]:
df_attend = df_attend.merge(
    df_users,
    left_on=['user_id'], 
    right_on=['user_id'],
    how='left')


In [20]:
display(df_attend)

### Сохраняем обогащенную таблицу

In [21]:
df_attend.to_csv('_attend_features_direct3.csv')